In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pythainlp
import pythaispell
from gingerit.gingerit import GingerIt
from langdetect import detect
import nltk
from keras import *
from keras.utils import to_categorical
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
from stop_words import get_stop_words
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report
from pythainlp.corpus.common import thai_words
from pythainlp.tokenize import dict_trie
from pythainlp.tag.named_entity import ThaiNameTagger

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# import data
df = pd.read_csv('../Description_classification/Product_description.csv')
df.describe()

,Category
count,61.000000
mean,6.475410
std,4.064507
min,1.000000
25%,3.000000
50%,6.000000
75%,10.000000
max,13.000000


In [3]:
df.head()

,Product,Description,Category
0,INOVO,INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ...,1
1,Pinno,"ขนาดหน้าจอ 2.4 นิ้ว,ระบบ Dual SIM (2 ซิม),รองร...",1
2,R9,มือถือจอหยดน้ำหน้าจอใหญ่สุดและสเปคแท้คุ้มค่าที...,1
3,J5pro (OEM),สินค้าลดล้างสต๊อกประกัน 7 วันเท่านั้น ไม่รองรั...,1
4,J9 Plus,J9 Plusมือถือ OEM ตอบโจทย์ทางเลือกสำหรับงบจำกั...,1


In [4]:
x = df['Description'].values
y = df['Category'].values

In [5]:
y.shape

(61,)

In [6]:
x[0]

'INOVO I 812 V1 (Ram:3GB/Rom:32GB) รับประกัน 1 ปี ,จอ 6.0" QHD  Quad-Core Processor Ram 3 GB/Rom 32 GB แบต 2500 mAh Battery ,5.0 MP+8.0MP Pixel , Face uniock, Android 8'

In [7]:
# custom dictionary
new_words = {"ขนาดหน้าจอ", "อะแดปเตอร์แปลงไฟ", "กล้องหน้า","วีดิโอคอล","บลูทูธ"}
words = new_words.union(thai_words())

custom_dictionary = dict_trie(words)

In [90]:
import re
import string

# word correction for EN
#parser = GingerIt()

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))
p_stemmer = PorterStemmer()

# clean the text
def clean_msg(msg): 
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    msg = re.sub(r'✅','',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

# split to words from text
def split_word(text):
    words = word_tokenize(text,engine='newmm',custom_dict=custom_dictionary,keep_whitespace=False)

    # Remove stop words TH and EN
    words = [i for i in words if not i in th_stop and not i in en_stop]
    
    # รากศัพท์
    # EN
    words = [p_stemmer.stem(i) for i in words]
    
    # TH
    tokens_temp=[]
    for i in words:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            if(w_syn[0].lemma_names('tha')[0]=='แบต'):
                tokens_temp.append(w_syn[0].lemma_names('tha')[1])
            elif(w_syn[0].lemma_names('tha')[0].isdigit()):
                tokens_temp.append(w_syn[0].lemma_names('tha')[2])
            else:
                tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    words = tokens_temp
    
    # delete whitespace
    #words = [i for i in words if not ' ' in i]

    return words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [91]:
clean_text = [clean_msg(text) for text in x]

In [92]:
word_list = [split_word(txt) for txt in clean_text]
print(word_list)

[['inovo', 'I', '812', 'วี', '1', 'ram', '3', 'gbrom', '32', 'GB', 'รับประกัน', '1', 'ปี', 'จอ', '60', 'qhd', 'quadcor', 'processor', 'ram', '3', 'gbrom', '32', 'GB', 'แบตเตอรี่', '2500', 'mah', 'batteri', '50', 'MP', '80', 'MP', 'pixel', 'face', 'uniock', 'android', '8'], ['ขนาดหน้าจอ', '24', 'นิ้ว', 'ระบบ', 'dual', 'sim', '2', 'ซิม', 'ค้ำ', 'SD', 'card', '8', 'GB', 'ปุ่ม', 'สะใจ', 'กด', 'เห็นชัด', 'ช่อง', 'สอด', 'หูฟัง', 'ขนาด', '35', 'mm', 'ฟัง', 'ค์ชั่น', 'เครื่อง', 'MP', '3', 'MP', '4', 'FM', 'กล้อง', 'อัดเทป', 'นาฬิกาปลุก', 'ไฟฉาย', 'ภาษา', 'ค้ำ', 'english', 'russian', 'french', 'spanish', 'indonesiamelayu', 'thai', 'ความถี่', 'dual', 'band', 'gsm', '900', '1800', 'mhz'], ['มือถือ', 'จอ', 'หยดน้ำ', 'หน้าจอ', 'สเปค', 'คุ้มค่า', 'ตลาด', 'ตอนนี้', 'หน้า', 'จอแสดงผล', '626', 'นิ้ว', 'ระบบปฏิบัติการ', 'android', 'ของแท้', 'ดี', 'สำหรับ', 'ธุรกรรม', 'การเงิน', 'ปลอดภัย', 'No', 'root', 'googl', 'play', 'protect', 'ป้องกัน', 'แอ', 'ป', 'ทำอันตราย', 'ซอ', 'ฟแวร์', 'อัพเดท', 'ลอยแพ', 'ฝา',

In [33]:
from pythainlp.tag import pos_tag_sents

pos_pud = pos_tag_sents(word_list, corpus='pud')

In [20]:
pos_tag_sents(word_list, corpus='orchid_ud')

[[('inovo', 'NOUN'),
  ('I', 'NOUN'),
  ('812', 'NUM'),
  ('วี', 'NOUN'),
  ('1', 'NUM'),
  ('ram', 'NOUN'),
  ('3', 'NUM'),
  ('gbrom', 'NOUN'),
  ('32', 'NUM'),
  ('GB', 'NOUN'),
  ('รับประกัน', 'NOUN'),
  ('1', 'NUM'),
  ('ปี', 'NOUN'),
  ('จอ', 'NOUN'),
  ('60', 'NUM'),
  ('qhd', 'NOUN'),
  ('quadcor', 'NOUN'),
  ('processor', 'NOUN'),
  ('ram', 'NOUN'),
  ('3', 'NUM'),
  ('gbrom', 'NOUN'),
  ('32', 'NUM'),
  ('GB', 'NOUN'),
  ('แบตเตอรี่', 'NOUN'),
  ('2500', 'NUM'),
  ('mah', 'NOUN'),
  ('batteri', 'NOUN'),
  ('50', 'NUM'),
  ('MP', 'NOUN'),
  ('80', 'NUM'),
  ('MP', 'NOUN'),
  ('pixel', 'NOUN'),
  ('face', 'NOUN'),
  ('uniock', 'NOUN'),
  ('android', 'NOUN'),
  ('8', 'NUM')],
 [('ขนาดหน้าจอ', 'NOUN'),
  ('24', 'NUM'),
  ('นิ้ว', 'NOUN'),
  ('ระบบ', 'NOUN'),
  ('dual', 'NOUN'),
  ('sim', 'NOUN'),
  ('2', 'NUM'),
  ('ซิม', 'NOUN'),
  ('ค้ำ', 'NOUN'),
  ('SD', 'NOUN'),
  ('card', 'NOUN'),
  ('8', 'NUM'),
  ('GB', 'NOUN'),
  ('ปุ่ม', 'NOUN'),
  ('สะใจ', 'NOUN'),
  ('กด', 'VERB'),
  

In [32]:
pos_tag_sents(word_list)

[[('inovo', 'NCMN'),
  ('I', 'NCMN'),
  ('812', 'NCNM'),
  ('วี', 'CMTR'),
  ('1', 'NCNM'),
  ('ram', 'NCMN'),
  ('3', 'NCNM'),
  ('gbrom', 'NCMN'),
  ('32', 'NCNM'),
  ('GB', 'NCMN'),
  ('รับประกัน', 'CNIT'),
  ('1', 'DCNM'),
  ('ปี', 'CMTR'),
  ('จอ', 'CNIT'),
  ('60', 'DCNM'),
  ('qhd', 'NCMN'),
  ('quadcor', 'NCMN'),
  ('processor', 'NCMN'),
  ('ram', 'NCMN'),
  ('3', 'NCNM'),
  ('gbrom', 'NCMN'),
  ('32', 'NCNM'),
  ('GB', 'NCMN'),
  ('แบตเตอรี่', 'NCMN'),
  ('2500', 'NCNM'),
  ('mah', 'NCMN'),
  ('batteri', 'CMTR'),
  ('50', 'DCNM'),
  ('MP', 'CMTR'),
  ('80', 'DCNM'),
  ('MP', 'CMTR'),
  ('pixel', 'NCMN'),
  ('face', 'NCMN'),
  ('uniock', 'NCMN'),
  ('android', 'NCMN'),
  ('8', 'NCNM')],
 [('ขนาดหน้าจอ', 'NCMN'),
  ('24', 'DCNM'),
  ('นิ้ว', 'CMTR'),
  ('ระบบ', 'NCMN'),
  ('dual', 'NCMN'),
  ('sim', 'NCMN'),
  ('2', 'NCNM'),
  ('ซิม', 'CMTR'),
  ('ค้ำ', 'CNIT'),
  ('SD', 'NCMN'),
  ('card', 'NCMN'),
  ('8', 'DCNM'),
  ('GB', 'NCMN'),
  ('ปุ่ม', 'CNIT'),
  ('สะใจ', 'NCMN'),
  ('ก

In [96]:
filtered = []
for idx,value in enumerate(pos_pud):
    tmp = []
    for p0,p1 in value:
        tmpp = []
        if(p1=='NUM') or (p1=='NOUN'):
            tmpp.append(p0)
            tmpp.append(p1)
            tmp.append(tmpp)
    print(tmp)
    
           
        

[['I', 'NOUN'], ['812', 'NUM'], ['วี', 'NOUN'], ['1', 'NUM'], ['ram', 'NOUN'], ['3', 'NUM'], ['gbrom', 'NOUN'], ['32', 'NUM'], ['GB', 'NOUN'], ['1', 'NUM'], ['ปี', 'NOUN'], ['จอ', 'NOUN'], ['60', 'NUM'], ['qhd', 'NOUN'], ['quadcor', 'NOUN'], ['processor', 'NOUN'], ['ram', 'NOUN'], ['3', 'NUM'], ['gbrom', 'NOUN'], ['32', 'NUM'], ['GB', 'NOUN'], ['แบตเตอรี่', 'NOUN'], ['2500', 'NUM'], ['batteri', 'NOUN'], ['50', 'NUM'], ['MP', 'NOUN'], ['80', 'NUM'], ['MP', 'NOUN'], ['pixel', 'NOUN'], ['face', 'NOUN'], ['uniock', 'NOUN'], ['android', 'NOUN'], ['8', 'NUM']]
[['ขนาดหน้าจอ', 'NOUN'], ['24', 'NUM'], ['นิ้ว', 'NOUN'], ['ระบบ', 'NOUN'], ['dual', 'NOUN'], ['sim', 'NOUN'], ['2', 'NUM'], ['ซิม', 'NOUN'], ['SD', 'NOUN'], ['card', 'NOUN'], ['8', 'NUM'], ['GB', 'NOUN'], ['สะใจ', 'NOUN'], ['ช่อง', 'NOUN'], ['สอด', 'NOUN'], ['หูฟัง', 'NOUN'], ['ขนาด', 'NOUN'], ['35', 'NUM'], ['mm', 'NOUN'], ['เครื่อง', 'NOUN'], ['MP', 'NOUN'], ['3', 'NUM'], ['MP', 'NOUN'], ['4', 'NUM'], ['FM', 'NOUN'], ['อัดเทป', 'NOU

# Create model

In [64]:
tokens_list_j = [','.join(tkn) for tkn in word_list]
cvec = CountVectorizer(analyzer=lambda x:x.split(','))
c_feat = cvec.fit_transform(tokens_list_j)

In [30]:
cvec.vocabulary_

{'inovo': 530,
 'I': 285,
 '812': 223,
 'V': 315,
 '1': 11,
 'ram': 687,
 '3': 106,
 'gbrom': 478,
 '32': 114,
 'GB': 279,
 'รับประกัน': 1411,
 'ปี': 1263,
 'จอ': 998,
 '60': 194,
 'qhd': 682,
 'quadcor': 684,
 'processor': 671,
 'แบตเตอรี่': 1844,
 '2500': 90,
 'mah': 578,
 'batteri': 356,
 '50': 161,
 'MP': 293,
 '80': 219,
 'pixel': 659,
 'face': 448,
 'uniock': 783,
 'android': 337,
 '8': 218,
 'ขนาดหน้าจอ': 905,
 '24': 81,
 'นิ้ว': 1198,
 'ระบบ': 1399,
 'dual': 428,
 'sim': 725,
 '2': 64,
 'ซิม': 1055,
 'ค้ำ': 991,
 'SD': 306,
 'card': 378,
 'ปุ่มกด': 1265,
 'สะใจ': 1524,
 'กด': 836,
 'เห็นชัด': 1807,
 'ช่อง': 1041,
 'สอด': 1515,
 'หูฟัง': 1613,
 'ขนาด': 904,
 '35': 124,
 'mm': 595,
 'ฟัง': 1326,
 'ค์ชั่น': 992,
 'เครื่อง': 1690,
 '4': 135,
 'FM': 277,
 'กล้อง': 861,
 'อัดเทป': 1643,
 'นาฬิกาปลุก': 1193,
 'ไฟฉาย': 1937,
 'ภาษา': 1339,
 'english': 439,
 'russian': 702,
 'french': 469,
 'spanish': 740,
 'indonesiamelayu': 527,
 'thai': 767,
 'ความถี่': 948,
 'band': 355,
 'gsm': 498

In [31]:
len(cvec.vocabulary_)

1971

In [34]:
tvec = TfidfVectorizer(analyzer=lambda x:x.split(','))
t_feat = tvec.fit_transform(tokens_list_j)

In [35]:
len(tvec.idf_)

1971

In [60]:
## กำหนดค่าคงที่ต่างๆ ซึ่งสามารถปรับจูนได้เพื่อประสิทธิภาพที่ดีขึ้น ในภายหลัง
embed_size = 300 # ชนาด dimension ของ word vectors
max_features = 100000 # จำนวนคำศัพท์ที่เราจะให้ model รู้จัก
maxlen = 70 # กำหนดให้ความยาวของทุกประโยคเท่ากันที่ 70

In [69]:
# Define model
lstm = models.Sequential()
lstm.add(layers.Embedding(max_features,embed_size,input_shape=(maxlen,)))
lstm.add(layers.LSTM(128))
lstm.add(layers.Dense(128,activation='relu'))
lstm.add(layers.Dense(14,activation='softmax'))

TypeError: The added layer must be an instance of class Layer. Found: Tensor("Encoder_input_3:0", shape=(None, 61, 1, 300), dtype=float32)

In [63]:
# Compile model
lstm.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy',metrics=['acc'])
lstm.summary()

ValueError: This model has not yet been built. Build the model first by calling build() or calling fit() with some data. Or specify input_shape or batch_input_shape in the first layer for automatic build. 

In [56]:
# Fit model
his =  lstm.fit(x,y,epochs=5)

ValueError: Error when checking input: expected embedding_5_input to have 2 dimensions, but got array with shape (61, 1, 300)

In [ ]:
# Define an evaluation function to print the evaluation result
def evaluation_report(model,features,labels):
    
    # Calculate result
    result = model.evaluate(features,labels,verbose=False)
    
    # Predict and convert into a class
    pred_class = model.predict(features).argmax(axis=1)
    
    # Show report
    print(confusion_matrix(labels,pred_class))
    print(classification_report(labels,pred_class))
    print("Loss: %s Accuracy: %s" %(result[0],result[1]))
    
    return pred_class

In [ ]:
evaluation_report(model,t_feat,y)